In [ ]:
%load_ext autoreload
%autoreload 2

import pickle
import numpy as np
import sympy as smp
from tqdm import tqdm
from sympy import Symbol, symbols
import centrex_TlF as centrex
import matplotlib.pyplot as plt
from julia import Main

### Papermill parameters

In [ ]:
Γ = 2*np.pi*1.56e6

transition_type = "R"
ground_J = 1
excited_J = 2
excited_F = 2
excited_F1 = 3/2
system_J = [1,3]

laser_power = 23.8e-3 # power in W
σlaser = 0.000914
npasses = 9
multipass_offset = 0.01
distance_between_passes = 0.03556/npasses
power_loss_per_pass = 20.2/23.8

vcm = 150

ωp = 0.65*Γ

β = 3.8
ωphase = 0.0

scan_range = [-10,10.25,0.25]

nprocs = 7

In [ ]:
trans = centrex.transitions.LaserTransition(
    f"{transition_type}{ground_J}",
    F1 = excited_F1,
    F = excited_F
)
trans

In [ ]:
excited_state = centrex.states.QuantumSelector(
                    J=excited_J, F=excited_F, F1=excited_F1, 
                    electronic = 'B', P = trans.excited_selector.P
)
print(f"excited state : {excited_state}")

# Initial setup
## Involved states

In [ ]:
gnd = centrex.states.QuantumSelector(J=system_J, electronic = 'X')
exc = excited_state

gnd_laser = centrex.states.QuantumSelector(J=trans.ground_selector.J, electronic = 'X')
exc_laser = excited_state

In [ ]:
syspars = centrex.states.SystemParameters(
    nprocs  = nprocs,
    Γ       = Γ,
    X  = gnd,
    B = [exc]
)

## Parameters

In [ ]:
odepars = centrex.lindblad.odeParameters(
    Ωl = "Ωl0 * phase_modulation(t, β, ωphase) * multipass_2d_intensity(vz*t, y0+vy*t, ampl, xlocs, ylocs, σzlaser, σylaser)",
    Ωl0     = 1*syspars.Γ,    # Rabi frequency of the laser [rad/s]
    δl      = 0.0,              # detuning of the laser [rad/s]
    
    # laser beam properties
    σzlaser = σlaser * np.sqrt(2),
    σylaser = σlaser * np.sqrt(2),
    
    # laser multipass
    ampl = [np.sqrt(power_loss_per_pass**n) for n in range(npasses)],
    xlocs = [(n-1)*distance_between_passes + multipass_offset for n in centrex.utils.light.multipass_prism_order(npasses)],
    ylocs = npasses*[0.0],
    
    # laser phase modulation
    ωphase = ωphase,       # laser phase modulation frequency [rad/s]
    β      = β,             # laser phase modulation depth [rad]

    # laser polarization switching
    ωp = ωp,           # polarization switching frequency [rad/s]
    φp = 0.0,                 # polarization switching phase [rad]
    Pl  = "sin(ωp*t + φp)",
    Plz = "Pl>0",
    Plx = "Pl<=0",
    Ply = 0.0,
    
    # molecules
    y0 = 0.,                   # molecule start y position [m]
    vz = vcm,                 # longitudinal molecular velocity [m/s]
    vy = 0.,                   # molecule vy [m/s]
)


## Transitions

In [ ]:
transitions = [
    centrex.couplings.TransitionSelector(
        ground = 1*centrex.states.generate_coupled_states_ground_X(gnd_laser),
        excited = 1*centrex.states.generate_coupled_states_excited_B(exc_laser),
        polarizations        = [[0,0,1],[1,0,0],[0,1,0]],
        polarization_symbols = symbols("Plz     Plx     Ply"),
        Ω = smp.Symbol('Ωl', complex = True),
        δ = smp.Symbol('δl'),
        description = "laser transition",        
    ),
]

In [ ]:
transition_name = centrex.transitions.identify_transition(
    transitions[0].ground[0],
    transitions[0].excited[0]
)
transition_name = transition_name.replace(",","").replace("/","_")
transition_name

## Generate System

In [ ]:
%%time
obe_system = centrex.lindblad.setup_OBE_system_julia(syspars, odepars, transitions, verbose=True, full_output=True, 
                                                     qn_compact=centrex.states.QuantumSelector(J=3, electronic = 'X'))

# Simulations
## Single trajectory for population in each HFS separately

In [ ]:
Ωl0 = centrex.utils.light.calculate_rabi_from_power_gaussian_beam(
                        laser_power,
                        obe_system.couplings[0]['main coupling'],
                        σlaser,
                        σlaser
    ).real
print(f"Ωl0 : {Ωl0 / syspars.Γ:.2f} Γ")

In [ ]:
odepars.Ωl0 = Ωl0
odepars.ωphase = ωphase
odepars.ωp = ωp

In [ ]:
nphotons = []
res = []
for idx, state in tqdm(enumerate(obe_system.QN), 
                       total = centrex.utils.J_levels(
                           trans.ground_selector.J
                       )
                      ):
    state = state.find_largest_component()
    if (state.J == 1) & (state.electronic_state == 'X'):
        ρ = centrex.utils.generate_population_states([idx], len(obe_system.QN))
        t_array, results = centrex.lindblad.do_simulation_single(odepars, (0,50e-3/odepars.vz+10e-6), ρ)
        res.append({'t': t_array, 'data': results})
        nphotons.append(
            np.trapz(results[exc.get_indices(obe_system.QN),:].sum(axis = 0), t_array)*syspars.Γ
        )

In [ ]:
for idx, state in tqdm(enumerate(obe_system.QN), 
                       total = centrex.utils.J_levels(
                           trans.ground_selector.J
                       )
                      ):
    state = state.find_largest_component()
    
    if (state.J == 1) & (state.electronic_state == 'X'):
        t_array, results = res[idx].values()
        _t = np.linspace(t_array.min(), t_array.max(), 201)
        multipass = np.abs(odepars.get_parameter_evolution(_t, "Ωl").real)
        
        fig, ax = plt.subplots(figsize = (10,6)) 
        ax.plot(t_array*1e6, results.T, lw = 2);
        ax.plot(_t*1e6, multipass/multipass.max()*0.8, 'k--', lw = 2,
                label = 'multipass')
        ax.set_xlabel('time [μs]')
        ax.set_ylabel('population')
        ax.set_title(f"{state}")
        ax.legend(fontsize = 14)
        ax.grid(True)

### Calculate Branching ratios

In [ ]:
branch = {}
for idx,i,j in zip(*np.nonzero(obe_system.C_array)):
    g = obe_system.QN[i].find_largest_component()
    e = obe_system.QN[j].find_largest_component()
    if trans.ground_selector.J == g.J:
        br = (obe_system.C_array[idx,i,j]**2 / syspars.Γ).real
        key = f"{e.J}, {e.F1}, {e.F}, {e.mF}"
        if branch.get(key) is None:
            branch[key] = br
        else:
            branch[key] += br
branch

### Save Results to pickle file

In [ ]:
with open(f"{transition_name}.pkl", "wb") as f:
    pickle.dump(
        dict(
            QN = obe_system.QN,
            results = res,
            nphotons = nphotons,
            branching = branch
        ),
        f
    )

## Detuning Scan

In [ ]:
odepars.Ωl0 = Ωl0
odepars.ωphase = ωphase
odepars.ωp = ωp

tspan = (0,1000e-6)

δl_scan = np.arange(*scan_range)*syspars.Γ

parameters = ["δl"]
values = [δl_scan]

output_func = centrex.lindblad.setup_state_integral_calculation(
                states = exc.get_indices(obe_system.QN, mode = "julia"),
                nphotons = True
            )
cb = centrex.lindblad.setup_discrete_callback_terminate(odepars, "vz*t >= 50e-3")

In [ ]:
scan_results = []
for idx, state in tqdm(enumerate(obe_system.QN), 
                       total = centrex.utils.J_levels(
                           trans.ground_selector.J
                       )
                      ):
    state = state.find_largest_component()
    if (state.J == 1) & (state.electronic_state == 'X'):
        ρ = centrex.utils.generate_population_states([idx], len(obe_system.QN))

        ens_prob = centrex.lindblad.setup_problem_parameter_scan(
                                odepars, tspan, ρ, parameters, values, 
                                dimensions = 1,
                                output_func = output_func,
                                zipped = True
                    )

        centrex.lindblad.solve_problem_parameter_scan(ensemble_problem_name = ens_prob, saveat = 1e-7,
                                                     callback = cb)
        scan_results.append(centrex.lindblad.get_results_parameter_scan())

In [ ]:
for idx, state in tqdm(enumerate(obe_system.QN), 
                       total = centrex.utils.J_levels(
                           trans.ground_selector.J
                       )
                      ):
    state = state.find_largest_component()
    
    if (state.J == 1) & (state.electronic_state == 'X'):
        results = scan_results[idx]
        
        fig, ax = plt.subplots(figsize = (8,5))
        ax.plot(δl_scan/syspars.Γ, results, '-', lw = 2)
        ax.set_xlabel("δ [Γ]")
        ax.set_label("#photons")
        ax.set_title(f"{state}")
        ax.grid(True)